In [19]:
#CT image reconstruct through the FBP algorithm
#Written by Ren-Qi Pan
import math
import os
from pydicom import dcmread, dcmwrite
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [20]:
_pth = r'F:\EPID Images\sphere\854320165'
_pth = r'D:\CMC\pyprojects\radio_therapy\dose-3d\dataset\2024-06-08 09-51-34-MV Dosimetry-6x\854350196'
_pth = r'C:\Users\Pintu\OneDrive - Christian Medical College\Fabricated Phantom Rotational RT\EPID_Calibration on Fabricated Phantom 10x10_Extracted'
_pth = r"C:\Users\Pintu\OneDrive - Christian Medical College\Fabricated Phantom Rotational RT\Calibration & Test_13.07.24_Extracted\TEST"
_folders = os.listdir(_pth)

In [21]:
PI=math.pi

def filter_SL(N,d):
    fh_SL=np.zeros(N)
    for k1 in range(0,N,1):
        fh_SL[k1]=-2.0/(PI*PI*d*d*(4*(k1-N/2.0)**2-1))
    return fh_SL

def nearestPowerOf2(N):
    # Calculate log2 of N
    a = int(math.log2(N))
    if 2**a == N:
        return N
    return 2**(a + 1)

In [22]:
from scipy.fft import fft, ifft

def Fun_Weigth_Projection(projection_beta,SOD,delta_dd):
    Nrows,Ncolumns=projection_beta.shape
    dd_column=delta_dd*np.arange(-Ncolumns/2+0.5,(Ncolumns/2+1)-0.5,1.0)#coordinate of detector cell in horizontal 
    dd_row=delta_dd*np.arange(-Nrows/2+0.5,(Nrows/2+1)-0.5,1.0)#coordinate of detector cell in vertical
    dd_row2D,dd_column2D=np.meshgrid(dd_row,dd_column,indexing='ij')
    weighted_projection=projection_beta*SOD/np.sqrt(SOD*SOD+np.power(dd_row2D,2.0)+np.power(dd_column2D,2.0))
    return weighted_projection

def optimize_convolution(weighted_projection, fh_RL):
    Nrows, Ncolumns = weighted_projection.shape
    Nfft = nearestPowerOf2(2 * Ncolumns - 1)
    fh_RL_padded = np.zeros(Nfft)
    fh_RL_padded[:len(fh_RL)] = fh_RL / 2.0  # Scale and pad filter response
    
    fh_RL_fft = fft(fh_RL_padded)  # FFT of the filter response
    
    # Zero pad the input projection for FFT
    projection_padded = np.zeros((Nrows, Nfft))
    projection_padded[:, :Ncolumns] = weighted_projection

    # Perform FFT on each row of the projection
    projection_fft = fft(projection_padded, axis=1)
    
    # Element-wise multiplication in the frequency domain
    convoluted_freq = projection_fft * fh_RL_fft
    
    # Perform the inverse FFT to get back to the time domain
    convoluted_time = ifft(convoluted_freq, axis=1).real
    
    # Slice to obtain the result with the correct dimensions
    filtered_projection = convoluted_time[:, :Ncolumns]
    
    return filtered_projection


def Fun_Filter_Projection(weighted_projection,fh_RL):
    Nrows,Ncolumns=weighted_projection.shape
    Nfft=nearestPowerOf2(2*Ncolumns-1)
    filtered_projection=np.zeros((Nrows,Ncolumns))
    for row in range(Nrows):
        projection_row=weighted_projection[row,:]
        zeros_pad=np.zeros(Nfft-Ncolumns)
        projection_row=np.concatenate((projection_row,zeros_pad))
        convoluted_time=np.convolve(projection_row,fh_RL/2.0,mode='same')
        convoluted_time=convoluted_time[:Ncolumns]
        filtered_projection[row,:]=convoluted_time
    return filtered_projection

def Fun_BackProjection(filtered_projection,SOD,beta_num,beta_m,delta_dd,Nimage):
    Nrows,Ncolumns=filtered_projection.shape
    MX, MZ=Nimage,int(Nimage*Nrows/Ncolumns)
    #roi[0], roi[1]: the min and max of x-axis and y-axis
    #roi[2],roi[3]: the min and max of z-axis
    
    roi=delta_dd*np.array([-Ncolumns/2.0+0.5,Ncolumns/2.0-0.5,-Nrows/2.0+0.5,Nrows/2.0-0.5])
    hx=(roi[1]-roi[0])/(MX-1) #interval of x-axis and y-axis
    xrange=roi[0]+hx*np.arange(0,MX) #coordinate vector of x-axis and y-axis 
    hy=(roi[3]-roi[2])/(MZ-1) #interval of z-axis
    yrange=roi[2]+hy*np.arange(0,MZ) #coordinate vector of z-axis 
    XX,YY,ZZ=np.meshgrid(xrange,xrange,yrange,indexing='ij')
    temp_rec=np.zeros((MX,MX,MZ)) #store back projection contribution
    U=(SOD+XX*np.sin(beta_m)-YY*np.cos(beta_m))/SOD
    a=(XX*np.cos(beta_m)+YY*np.sin(beta_m))/U
    xx=np.int32(np.floor(a/delta_dd)) #beam numbering,strart from -Ncolumns/2
    u1=a/delta_dd-xx  #the decimal part of a
    b=ZZ/U
    yy=np.int32(np.floor(b/delta_dd)) #beam numbering,strart from -Nrows/2
    u2=b/delta_dd-yy #the decimal part of b
    xx=xx+int(Ncolumns/2) #numbering from 0
    yy=yy+int(Nrows/2) #numbering from 0

    mask=np.where((xx >=0) & (xx< Ncolumns-1) & (yy >=0) & (yy<Nrows-1))
    xx=xx[mask]
    yy=yy[mask]
    u1=u1[mask]
    u2=u2[mask]
    # print(mask)
    temp=(1-u1)*(1-u2)*filtered_projection[yy,xx]+(1-u1)*u2*filtered_projection[yy+1,xx]+\
                 (1-u2)*u1*filtered_projection[yy,xx+1]+u1*u2*filtered_projection[yy+1,xx+1]
    temp_rec[mask]=temp_rec[mask]+temp/(np.power(U[mask],2))*2*PI/beta_num
    # print('backprojection, beta: ',round(beta_m*180/PI,1))
    return temp_rec


In [23]:
def ConeBeam_FDK_Backprojection(projection,fh_RL,beta,SOD,Nimage,delta_dd):
    # projection: projection data in 3D, (angle,row, colums)
    # fh_RL: R-L filter function
    # beta: rotate angles in degrees
    # SOD: source to rotation center distance in mm unit
    # delta_dd: interval of the virtual detector cell
    Ncolumns=projection.shape[2]
    Nrows=projection.shape[1]
    rec_image=np.zeros((Nimage,Nimage,int(Nimage*Nrows/Ncolumns)))
    beta_num=len(beta) #number of angles
    beta=beta*PI/180.0 # view angle in radian
    for m in tqdm(range(0, beta_num,1)):
        projection_beta=projection[m,:,:] #projection matrix at the beta angle
        weighted_projection=Fun_Weigth_Projection(projection_beta,SOD,delta_dd)
        filtered_projection=optimize_convolution(weighted_projection,fh_RL)
        rec_image=rec_image+Fun_BackProjection(weighted_projection,SOD,beta_num,beta[m],delta_dd,Nimage)
        # break
    return rec_image

In [24]:
_max_pixel = []
import pickle
for _f in tqdm(_folders):
    _cur_dir = os.path.join(_pth, _f, os.listdir(os.path.join(_pth, _f))[0])
    print(_cur_dir)
    
    _files = os.listdir(_cur_dir)
    
    dcm = dcmread(os.path.join(_cur_dir, _files[0]))
    shape = dcmread(os.path.join(_cur_dir, _files[0])).pixel_array.shape
    
    g_angle = []
    _images = np.zeros((len(_files),shape[0], shape[0]), dtype=np.uint16)
    prev = np.zeros((shape[0], shape[0]), dtype=np.uint16)
    curr = np.zeros((shape[0], shape[0]), dtype=np.uint16)
    _raw_datas = []
    for idx, _fname in enumerate(tqdm(_files)):
        raw = dcmread(os.path.join(_cur_dir, _fname))
        curr = raw.pixel_array
        
        _m = curr - prev
        
        if np.max(_m) > 10000:
            _images[idx, :, :] = _images[idx-1, :, :]
            g_angle.append(g_angle[idx-1])
        else:
            _images[idx, :, :] = curr - prev
            prev = curr
            g_angle.append(raw.GantryAngle)
            
        prev = curr


    g_angle = np.array(g_angle)
    sorted_inx = np.argsort(g_angle)

    sorted_images = np.zeros((len(_files),shape[0], shape[0]), dtype=np.uint16)

    for idx, val in enumerate(tqdm(sorted_inx)):
        sorted_images[idx, :, :] = _images[val, :, :]
        
    g_angle2 = g_angle[sorted_inx]
    _images = []
    
    
    SID = dcm.RTImageSID
    SAD = dcm.RadiationMachineSAD
    
    Ncolumns = sorted_images.shape[2]
    
    Nimage=100 #size of image
    beta=g_angle2 #rotate(view) angles
    SOD=SAD#source to origin distance, in unit mm
    SDD=SID #source to center of detector, in unit mm
    width=0.172 #size of detector cell, in unit mm
    delta_dd=width*SOD/SDD #interval of the virtual detector cell
    Nfft=nearestPowerOf2(2*Ncolumns-1) #number of points for FFT
    fh_RL=filter_SL(Nfft,delta_dd)  #get the filter kernel function
    #reconstruct the scanned object through the FBP algorithm
    rec_image=ConeBeam_FDK_Backprojection(sorted_images,fh_RL,beta,SOD,Nimage,delta_dd)
    
    _max_pixel.append(np.max(rec_image))
    _dump_file_pth = os.path.join(r'E:\CMC\pyprojects\radio_therapy\dose-3d\notebooks\M_TEST', f'{_f}.pkl')
    
    _dump_file = open(_dump_file_pth, 'wb')
    pickle.dump(rec_image, _dump_file)
    _dump_file.close()
    # print(np.max(rec_image))

  0%|          | 0/4 [00:00<?, ?it/s]

C:\Users\Pintu\OneDrive - Christian Medical College\Fabricated Phantom Rotational RT\Calibration & Test_13.07.24_Extracted\TEST\2024-07-13 16-07-28-MV Dosimetry-6x_8x8_Test1\854384206


 25%|██▌       | 1/4 [02:29<07:29, 149.84s/it]

C:\Users\Pintu\OneDrive - Christian Medical College\Fabricated Phantom Rotational RT\Calibration & Test_13.07.24_Extracted\TEST\2024-07-13 16-08-43-MV Dosimetry-6x_13x13_Test2\854384207


 50%|█████     | 2/4 [05:37<05:43, 171.84s/it]

C:\Users\Pintu\OneDrive - Christian Medical College\Fabricated Phantom Rotational RT\Calibration & Test_13.07.24_Extracted\TEST\2024-07-13 16-23-52-MV Dosimetry-6x_17x17_Test3\854384208


 75%|███████▌  | 3/4 [08:05<02:41, 161.22s/it]

C:\Users\Pintu\OneDrive - Christian Medical College\Fabricated Phantom Rotational RT\Calibration & Test_13.07.24_Extracted\TEST\2024-07-13 16-25-04-MV Dosimetry-6x_12x12_Test4\854384209


100%|██████████| 4/4 [10:41<00:00, 160.48s/it]
